In [1]:
# pip install praw

Note: you may need to restart the kernel to use updated packages.


In [1]:
import praw
import pandas as pd
import time
import re
from datetime import datetime

# Reddit API credentials
CLIENT_ID = "1OGPJ-KHnQSCWKGg-N5StQ"
CLIENT_SECRET = "w7JLrIPYRJlqxpPPys_syEWHsZ-UoA"
USER_AGENT = "Project:v1.0"

# Authenticate with Reddit
reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    user_agent=USER_AGENT
)

# Define subreddits and keywords
subreddits = ["environment", "climate", "sustainability", "science", "news"]
keywords = ["climate change", "global warming", "sustainability", "renewable energy", "carbon footprint"]

# Initialize variables for pagination
post_data = []
max_posts = 100000  # Increase dataset size for better prediction
batch_size = 100  # Number of posts per request

# Fetch historical posts with pagination
for sub in subreddits:
    subreddit = reddit.subreddit(sub)
    for keyword in keywords:
        try:
            # Get historical posts using `new` and `top`
            for post in subreddit.new(limit=max_posts):
                created_at = datetime.utcfromtimestamp(post.created_utc)
                year = created_at.year
                month = created_at.month
                quarter = (month - 1) // 3 + 1
                
                # Extract hashtags from title and text
                hashtags = re.findall(r"#\w+", post.title + " " + post.selftext)
                
                # Determine engagement level
                engagement_score = post.score + post.num_comments
                if engagement_score > 1000:
                    engagement_level = "High"
                elif engagement_score > 100:
                    engagement_level = "Medium"
                else:
                    engagement_level = "Low"
                
                post_data.append({
                    "Post_ID": post.id,
                    "Platform": "Reddit",
                    "Hashtag": ", ".join(hashtags) if hashtags else None,
                    "Content_Type": "Post",
                    "Likes": post.score,
                    "Comments": post.num_comments,
                    "Engagement_Level": engagement_level,
                    "Title": post.title,
                    "Text": post.selftext,
                    "Created_At": created_at.strftime("%Y-%m-%d %H:%M:%S"),
                    "Year": year,
                    "Month": month,
                    "Quarter": quarter,
                    "Subreddit": post.subreddit.display_name,
                    "Keyword": keyword
                })

            print(f"Collected {len(post_data)} posts so far from r/{sub} on '{keyword}'...")
            time.sleep(2)
        except Exception as e:
            print(f"Error scraping {sub} for '{keyword}': {e}")
            continue

# Convert to DataFrame
df_reddit = pd.DataFrame(post_data)

# Save to CSV
df_reddit.to_csv("reddit_historical_data.csv", index=False)
print(f"✅ Scraped {len(df_reddit)} Reddit posts successfully!")
